In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch.utils.tensorboard as tb
import preprocessing
from torch.utils.data.sampler import SequentialSampler
import seaborn as sns
import matplotlib.pyplot as plt
import os
import logging
torch.set_printoptions(profile="full")
import pandas as pd

In [2]:
#static parameters
train_batch_size = 170
val_batch_size = 170
input_size = 38
hidden_size = 128
num_layer = 2
output_size = 38
clip = 3

In [3]:
#get data from preprocessing.py
dataset_path = os.path.join(os.path.abspath('..'),'Dataset\\Clementi dataset\\Clementi dataset' )
network_input, network_output = preprocessing.preprocess_notes(dataset_path)
network_input, network_output = network_input.cuda(), network_output.cuda()
#print(network_input)
#print(network_output)

In [4]:
'''
data is highly unbalanced
'''
#sns.distplot(torch.tensor(network_output).cpu())

'\ndata is highly unbalanced\n'

In [5]:
'''
to make batch of equal sizes
Quick Fix
'''
network_input = network_input[: -117]
network_output = network_output[: -117]

print(network_input.shape)
print(network_output.shape)

torch.Size([8500, 50, 1])
torch.Size([8500])


In [6]:
'''
create Stacked LSTM model
'''
class Stacked_LSTM(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers, output_size):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first = True, dropout = 0.5)
        
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(hidden_size, output_size)
        
        
    def forward(self, x, hidden,batch_size):
        
        output, hidden = self.lstm(x, hidden)        
        #print(output.shape)
        
        # stack up lstm outputs
        output = output.contiguous().view(-1, self.hidden_size)
        #print(output.shape)
        
        output = self.dropout(output)
        output = self.linear(output)
        #print('Linear Output :-',output.shape)
        
        output = F.softmax(output, dim = 1)
        #print('SOFTMAX OUTPUT :--', output)
        
        
        # reshape to be batch_size first
        output = output.view(batch_size, -1)
        #print('Reshape to batch size first :-',output.shape)
        
        output = output[:, -x.shape[2]:] # get last batch of labels
        #print('Final Output :-',output)

        
        return output, hidden
    
    def hidden_init(self,batch_size):
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.num_layers, batch_size, self.hidden_size).zero_(),
          weight.new(self.num_layers, batch_size, self.hidden_size).zero_())
        return hidden

#initialize the weights of LSTM using Xavier initialization    
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)

    

In [7]:
'''
Divide the dataset into train/val 
'''
train_size = 0.8
indices = list(range(len(network_input)))
split = int(np.floor(train_size*len(network_input)))
train_idx, val_idx = indices[:split], indices[split:]

train_sampler = SequentialSampler(train_idx)
val_sampler = SequentialSampler(val_idx)

dataset = TensorDataset(network_input,network_output)
train_loader = DataLoader(dataset, batch_size= train_batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size= val_batch_size,sampler= val_sampler)

In [195]:
import torch.optim as optimizer

model = Stacked_LSTM(input_size,hidden_size,num_layer,output_size)
model.apply(init_weights)

criterion = nn.CrossEntropyLoss()
#optimizer = optimizer.AdamW(model.parameters())
optimizer = optimizer.RMSprop(model.parameters(), lr = 0.001, weight_decay = 0.01)


#make sure to transfer model to GPU after initializing optimizer
model.cuda()

Stacked_LSTM(
  (lstm): LSTM(38, 128, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=128, out_features=38, bias=True)
)

In [196]:
optimizer

RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.001
    momentum: 0
    weight_decay: 0.01
)

In [197]:
epochs = 10

for e in range(epochs):
    
    train_loss = 0
    val_loss = 0
    train_accuracy = 0
    val_accuracy = 0
    
    hidden = model.hidden_init(train_batch_size)
    #print(hidden[0].data.shape)
    
    #print('hidden[0].shape:- ',hidden[0].shape)
    
    for inputs,labels in train_loader:
        #print(inputs.shape)

        '''
        Creating new variables for the hidden state, otherwise
        we'd backprop through the entire training history
        '''
        h = tuple([each.data for each in hidden])
        

        # zero accumulated gradients
        model.zero_grad()

        # get the output from the model
        output, h = model.forward(inputs, h,train_batch_size)
        #print(output)
        
        top_p, top_class = output.topk(1, dim=1)
        logging.debug(' top probab {} top class {}'.format(top_p.view(-1, top_p.shape[0]), top_class.view(-1, top_p.shape[0])))

        #print('Labels Shape :-', (torch.max(labels, 1)[1]).shape)
    
        # calculate the loss and perform backprop
        #print('Labels Long :-', labels.long())
        loss = criterion(output,labels.long())
        
        loss.backward()
        
        #calculate training accuracy
        top_p, top_class = output.topk(1, dim=1)
        equals = top_class == labels.long().view(*top_class.shape)
        train_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        #nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        train_loss += loss.item()
        #print(train_loss)
              
    model.eval()
    for inputs, labels in val_loader:
                
        val_h = tuple([each.data for each in hidden])
        
        output, hidden = model.forward(inputs, val_h,val_batch_size)
        loss = criterion(output,labels.long())
        
        val_loss += loss.item()
        
        #calculate validation accuracy
        top_p, top_class = output.topk(1, dim=1)
        #logging.debug(output)
        #logging.debug('VALIDATION top probab {} VALIDATION top class {}'.format(top_p.view(-1, top_p.shape[0]), top_class.view(-1, top_p.shape[0])))

        #print('Top Class:- ',top_class)
        equals = top_class == labels.long().view(*top_class.shape)
        #print('Equals:- ', equals)
        val_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
    model.train()
    
    #Averaging losses
    train_loss = train_loss/len(train_loader)
    val_loss = val_loss/len(val_loader)
    val_accuracy = val_accuracy/len(val_loader)
    train_accuracy = train_accuracy/len(train_loader)
    
    print('Epoch: {}\tTrain Loss: {:.7f} \tVal Loss:{:.7f} \tTrain Acc: {:.7}% \tVal Acc: {:.7f}%'.format(e, train_loss, val_loss, train_accuracy*100,val_accuracy*100))      

Epoch: 0	Train Loss: 3.6357302 	Val Loss:3.6353582 	Train Acc: 10.36765% 	Val Acc: 11.4117651%
Epoch: 1	Train Loss: 3.6354549 	Val Loss:3.6352228 	Train Acc: 11.10294% 	Val Acc: 11.4117651%
Epoch: 2	Train Loss: 3.6353307 	Val Loss:3.6350822 	Train Acc: 11.10294% 	Val Acc: 11.4117651%
Epoch: 3	Train Loss: 3.6352083 	Val Loss:3.6349547 	Train Acc: 11.10294% 	Val Acc: 11.4117651%
Epoch: 4	Train Loss: 3.6351055 	Val Loss:3.6348539 	Train Acc: 11.10294% 	Val Acc: 11.4117651%
Epoch: 5	Train Loss: 3.6350260 	Val Loss:3.6347778 	Train Acc: 11.10294% 	Val Acc: 11.4117651%
Epoch: 6	Train Loss: 3.6349662 	Val Loss:3.6347203 	Train Acc: 11.10294% 	Val Acc: 11.4117651%
Epoch: 7	Train Loss: 3.6349212 	Val Loss:3.6346771 	Train Acc: 11.10294% 	Val Acc: 11.4117651%
Epoch: 8	Train Loss: 3.6348870 	Val Loss:3.6346441 	Train Acc: 11.10294% 	Val Acc: 11.4117651%
Epoch: 9	Train Loss: 3.6348610 	Val Loss:3.6346189 	Train Acc: 11.10294% 	Val Acc: 11.4117651%


### Music Genaration


In [ ]:
test_seq = torch.from_numpy(one_hot_encode(test_data, dict_size, seq_len, 1))
test_seq.shape

In [ ]:
#Predicting
test_hidden = model.hidden_init(test_seq.shape[0])
test_output,_ = model.forward(test_seq, test_hidden,test_seq.shape[0])

top_p, top_class = test_output.topk(1,dim =1)
top_class

In [ ]:
preprocessing.preprocess_notes()